# 환경 설정

In [1]:
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
# Colab에 필요한 라이브러리 설치
import requests
import pandas as pd
import time

In [8]:
def fetch_questions_with_accepted_answers(tag, api_key, pagesize=100, max_pages=100):
    questions_data = []

    url = "https://api.stackexchange.com/2.3/questions"
    for page in range(1, max_pages + 1):
        params = {
            "order": "desc",
            "sort": "activity",
            "site": "stackoverflow",
            "pagesize": pagesize,
            "page": page,
            "tagged": tag,
            "filter": "withbody",
            "key": api_key
        }
        response = requests.get(url, params=params)
        questions = response.json().get("items", [])

        for question in questions:
            # Check if the question has an accepted answer
            if question["is_answered"] and question.get("accepted_answer_id"):
                # Fetch the accepted answer
                accepted_answer = fetch_answer(question["accepted_answer_id"], api_key)
                if accepted_answer:
                    questions_data.append({
                        "question_title": question["title"],
                        "accepted_answer_body": accepted_answer
                    })
        time.sleep(1)  # API 요청 제한 준수

    # Create a DataFrame from the collected data
    df = pd.DataFrame(questions_data)
    return df

def fetch_answer(answer_id, api_key):
    url = f"https://api.stackexchange.com/2.3/answers/{answer_id}"
    params = {
        "order": "desc",
        "sort": "activity",
        "site": "stackoverflow",
        "filter": "withbody",
        "key": api_key
    }
    response = requests.get(url, params=params)
    items = response.json().get("items", [])

    if items:
        return items[0].get("body")  # Return the body of the accepted answer
    return None

# 실행 예시
api_key = "rl_q4m61iyCwBsGNL6sY4oHnVPVd"  # 여기에 API 키를 입력하세요
tag = "python"  # 원하는 태그를 지정하세요
df = fetch_questions_with_accepted_answers(tag, api_key)

# 데이터프레임을 CSV 파일로 저장
df.to_csv("qna.csv", index=False)
print("CSV 파일이 성공적으로 생성되었습니다!")


CSV 파일이 성공적으로 생성되었습니다!
